In [1]:
import torch
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

In [2]:
data = pd.read_csv('./data/balanced_sclaer_dataset_diabetes.csv')
data

,Unnamed: 0,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_012
0,0,0.000000,0.000000,1.000000,0.174419,1.000000,0.0,0.0,1.000000,1.000000,...,0.000000,0.250000,0.066667,0.000000,0.000000,0.000000,0.166667,0.600000,0.000000,0.0
1,1,0.000000,1.000000,1.000000,0.220930,1.000000,0.0,1.0,1.000000,1.000000,...,0.000000,0.500000,0.000000,0.000000,0.000000,1.000000,0.583333,0.800000,0.857143,2.0
2,2,0.000000,0.000000,1.000000,0.174419,0.000000,0.0,0.0,1.000000,0.000000,...,0.000000,0.500000,0.100000,0.000000,0.000000,0.000000,0.500000,0.600000,0.714286,0.0
3,3,0.000000,0.000000,1.000000,0.151163,1.000000,0.0,0.0,1.000000,1.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.500000,1.000000,0.857143,0.0
4,4,1.000000,1.000000,1.000000,0.209302,1.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.500000,1.000000,0.000000,0.000000,0.000000,0.250000,0.600000,1.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105286,105286,1.000000,1.000000,1.000000,0.250595,0.775591,0.0,0.0,0.000000,0.000000,...,0.224409,0.806102,0.462599,0.500000,1.000000,0.775591,0.750000,0.955118,0.728909,1.0
105287,105287,1.000000,1.000000,1.000000,0.243025,0.900157,0.0,0.0,1.000000,0.099843,...,0.000000,1.000000,1.000000,1.000000,0.099843,0.000000,0.591654,0.619969,0.014263,1.0
105288,105288,1.000000,0.699485,1.000000,0.330221,0.699485,0.0,0.0,0.300515,1.000000,...,0.000000,0.424871,0.000000,0.000000,0.000000,1.000000,0.525043,0.600000,1.000000,1.0
105289,105289,0.508129,0.508129,1.000000,0.250095,0.508129,0.0,0.0,0.491871,0.508129,...,0.000000,0.372968,0.016396,0.066667,0.000000,0.000000,0.500000,1.000000,0.929733,1.0


In [3]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.isnull().sum().sum()


0

In [4]:
y = data["Diabetes_012"].values
X = data.drop(["Diabetes_012"], axis=1).values

In [5]:
y = y.reshape(-1, 1)

In [6]:
# разбиваем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((89497, 21), (15794, 21), (89497, 1), (15794, 1))

In [7]:
batch_size = 256

In [8]:
# а теперь преобразуем обучающую выборку в объект Dataset
train_ds = TensorDataset(torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(y_train).type(torch.float32))
# для загрузки данных в ходе обучения мы создаем объект DataLoader на основе объекта Dataset
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [9]:
x_c, y_c = next(iter(train_dl))
x_c.shape, y_c.shape
# 128 - это указанный нами batch_size

(torch.Size([256, 21]), torch.Size([256, 1]))

In [10]:
test_ds = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [11]:
class MyClassifierModel(nn.Module):
    def __init__(self):
        super(MyClassifierModel, self).__init__()

        self.fc1 = nn.Linear(21, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 32)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(32, 3)
        # self.relu5 = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x= self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        # y = self.relu5(y)
        x = self.softmax(x)
        return x

In [12]:
model=MyClassifierModel()
print(model)

MyClassifierModel(
  (fc1): Linear(in_features=21, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=32, out_features=3, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [13]:
# определяем функцию потерь
loss = nn.NLLLoss()
# настраиваем оптимизатор и передаем туда параметры модели
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

In [14]:
epochs = 5
# цикл обучения (по эпохам)
for epoch in range(epochs):
    # за одну эпоху смотрим все батчи (по batch_size элементов)
    for x_b, y_b in train_dl:
        # делаем прямое распространение (получаем предсказание)
        outputs = model(x_b)
        # вычисляем значение функции потерь
        loss_value = loss(outputs, y_b)
        # делаем backward - вычисляются значения .grad у слоев модели
        loss_value.backward()
        # делаем шаг градиентного спуска с заданным у оптимизатора learning_rate
        optimizer.step()
        # зануляем .grad у слоев модели - для нового батча будем акумулировать новый .grad
        optimizer.zero_grad()

    # в конце эпохи выводим значение функции потерь для последнего рассмотренного батча
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

RuntimeError: 0D or 1D target tensor expected, multi-target not supported